In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dir_path = '/kaggle/input/major-vs-minor-guitar-chords/shords_dataset/'

In [ ]:
import torch 
import torchaudio
import IPython
from scipy import signal
from scipy.io.wavfile import read as read_wav
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
major_filenames = os.listdir('/kaggle/input/major-vs-minor-guitar-chords/shords_dataset/major')
minor_filenames = os.listdir('/kaggle/input/major-vs-minor-guitar-chords/shords_dataset/minor')

In [ ]:
def play_audio(path):
    sampling_rate, data=read_wav(path)
    return IPython.display.Audio(path, rate = sampling_rate)

In [ ]:
def print_spectrogram(path):
    sample_rate, data=read_wav(path)
    #stereo to mono
    mono = (data[:,0] + data[:,1]) / 2 
    plt.specgram(mono, Fs=sample_rate)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

In [ ]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, dir_path, major_filenames, minor_filenames, cut = 30000):
        self.major_filenames = major_filenames
        self.minor_filenames = minor_filenames
        self.dir_path = dir_path
        self.cut = cut
        
    def __len__(self):
        return len(self.major_filenames) + len(self.minor_filenames)      
    
    def __getitem__(self, idx):
        is_major =  idx < len(self.major_filenames)
        path = self.dir_path + ('major/' + self.major_filenames[idx] if is_major else 'minor/' + self.minor_filenames[idx - len(self.major_filenames)])
        waveform , _ = torchaudio.load(path)
        waveform = waveform[:,:self.cut].mean(axis = 0)
        specgram = torchaudio.transforms.Spectrogram()(waveform)
        sp_shape = specgram.shape
        return specgram.reshape(1, sp_shape[0], sp_shape[1]), is_major

In [ ]:

dataset = Dataset(
    dir_path, major_filenames, minor_filenames
)
train_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size = 32,
                                        shuffle = True)

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn = nn.BatchNorm2d(output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):    
        out = self.conv(x)
        out = self.pool(out)
        out = self.bn(out)
        out = self.relu(out)
        return out

class CNN(nn.Module):
    def __init__(self, dims, num_classes = 2):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList([])
        for i in range(1, len(dims)):
            self.layers.append(CNNBlock(input_dim = dims[i - 1], output_dim = dims[i]))
        
        self.gap = nn.AvgPool2d(kernel_size = (25, 18))
        self.fc = nn.Linear(dims[-1], num_classes)
        
        
    def forward(self, x):     
        for layer in self.layers:
            x = layer(x)
        out = self.gap(x)
        shape = out.shape
        out = out.reshape(shape[0], shape[1])
        out = self.fc(out)
        return out    

In [ ]:
model = CNN([1,32,64,128])

In [ ]:
epochs = 30

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
model.to(device)

for epoch in range(epochs):
    for idx, (data, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        data = data.to(device).float()
        label = label.to(device).float()
        pred = model(data)
        loss = criterion(pred, label.long())
        loss.backward()
        optimizer.step()
        if idx % 100 == 0:
            y_hat = torch.argmax(pred, dim = 1)
            correct = (y_hat == label).sum()
            print(f"Epoch {epoch} {idx}/{len(train_dataloader)} Loss = {loss.data:.03f}, acc = {correct / label.shape[0]:.02f}")

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/guitar_chord_model.pth')